In [1]:
# Step 0: install
!pip install --upgrade -q langchain langchain-community langchain-openai langchain-mcp-adapters faiss-cpu beautifulsoup4 unstructured sentence-transformers aiohttp pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
import os
import json
import asyncio
from aiohttp import web
from google.colab import userdata
from pyngrok import ngrok
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from tqdm import tqdm
from typing import List, Dict, Any
from dataclasses import dataclass
import aiohttp

In [3]:
# Set API keys and ngrok authtoken
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY4')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["NGROK_AUTH_TOKEN"] = userdata.get('NGROK_TOKEN')

In [28]:
# Define the Google Serper tool logic
async def google_serper_search(query: str) -> List[Dict[str, str]]:
    try:
        search = GoogleSerperAPIWrapper(k=10, type="search")
        # Use results method for site-specific searches
        results = search.results(query)
        organic = results.get("organic", [])
        return [{"title": item.get("title", ""), "link": item.get("link", "")} for item in organic[:10]]
    except Exception as e:
        return [{"error": f"Search failed: {str(e)}"}]

In [31]:
# import asyncio
# async def test_search_function():
#     test_urls = [
#         'aalas.org: How to prevent Rabies?',
#         'jstor.org: How to prevent Rabies?',
#         'avmajournals.avma.org: How to prevent Rabies?',
#         'animalmicrobiome.biomedcentral.com: How to prevent Rabies?',
#         'irishvetjournal.biomedcentral.com: How to prevent Rabies?',
#         'bmcvetres.biomedcentral.com: How to prevent Rabies?',
#         'animaldiseases.biomedcentral.com: How to prevent Rabies?',
#         'actavetscand.biomedcentral.com: How to prevent Rabies?',
#         'sciendo.com: How to prevent Rabies?',
#         'canadianveterinarians.net: How to prevent Rabies?',
#         'parasite-journal.org: How to prevent Rabies?',
#         'sciencedirect.com: How to prevent Rabies?',
#         'microbiologyresearch.org: How to prevent Rabies?',
#         'jvas.in: How to prevent Rabies?',
#     ]

#     print("Testing google_serper_search function:")
#     for url_query in test_urls:
#         print(f"\nSearching for: {url_query}")
#         try:
#             # Assuming google_serper_search is defined in a previous cell and is accessible
#             results = await google_serper_search(url_query)
#             print("Results:")
#             for item in results:
#                 print(f"  Title: {item.get('title', 'N/A')}")
#                 print(f"  Link: {item.get('link', 'N/A')}")
#                 if "error" in item:
#                     print(f"  Error: {item['error']}")
#         except Exception as e:
#             print(f"An error occurred during search for {url_query}: {e}")

# # Run the test function
# await test_search_function()

Testing google_serper_search function:

Searching for: aalas.org: How to prevent Rabies?
Results:
  Title: Rabies Prevention and Control - CDC
  Link: https://www.cdc.gov/rabies/prevention/index.html
  Title: Rabies | American Veterinary Medical Association
  Link: https://www.avma.org/resources-tools/one-health/rabies
  Title: Rabies Prevention | doh - DC Health
  Link: https://dchealth.dc.gov/service/rabies-prevention
  Title: AALAS.org
  Link: https://www.aalas.org/
  Title: Rabies - StatPearls - NCBI Bookshelf
  Link: https://www.ncbi.nlm.nih.gov/books/NBK448076/
  Title: Human rabies prevention and management
  Link: https://www.who.int/activities/human-rabies-prevention-and-management
  Title: Rabies - CDPH - CA.gov
  Link: https://www.cdph.ca.gov/Programs/CID/DCDC/pages/rabies.aspx
  Title: About Rabies - CDC
  Link: https://www.cdc.gov/rabies/about/index.html
  Title: [PDF] How to Prevent Exposure to Rabies
  Link: https://epi.dph.ncdhhs.gov/cd/lhds/manuals/rabies/docs/public_i

In [9]:
# MCP server routes
async def handle_jsonrpc(request):
    try:
        data = await request.json()
        method = data.get('method')
        request_id = data.get('id')

        if method == 'get_tool_schema':
            schema = {
                "name": "google_serper_search",
                "description": "Search the web using Google Serper API and return relevant URLs.",
                "parameters": {
                    "type": "object",
                    "properties": {"query": {"type": "string"}},
                    "required": ["query"]
                }
            }
            response = {"jsonrpc": "2.0", "result": schema, "id": request_id}
        elif method == 'google_serper_search':
            query = data.get('params', {}).get('query', '')
            results = await google_serper_search(query)
            response = {"jsonrpc": "2.0", "result": results, "id": request_id}
        else:
            response = {"jsonrpc": "2.0", "error": {"code": -32601, "message": "Method not found"}, "id": request_id}

        return web.json_response(response)
    except json.JSONDecodeError:
        return web.json_response(
            {"jsonrpc": "2.0", "error": {"code": -32700, "message": "Parse error"}, "id": None}, # Use None instead of null
            status=400
        )

async def handle_get(request):
    return web.Response(text="MCP server: Use POST for JSON-RPC requests")

# Start the aiohttp server
from aiohttp import web
app = web.Application()
app.add_routes([
    web.get('/', handle_get),
    web.post('/', handle_jsonrpc)
])

async def start_server():
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, '0.0.0.0', 5555)
    await site.start()
    print(f"MCP server running on port 5555")

# Run the server in a background task
asyncio.create_task(start_server())

# Wait for server to initialize
async def wait_for_server():
    await asyncio.sleep(5)

await wait_for_server() # Call with await

MCP server running on port 5555


In [10]:
# Expose the server via ngrok
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])
http_tunnel = ngrok.connect(5555, bind_tls=True)
public_url = http_tunnel.public_url
print(f"MCP server exposed at: {public_url}")

MCP server exposed at: https://1488386b9843.ngrok-free.app


In [12]:
# Simple tool class to mimic LangChain tool
@dataclass
class SimpleTool:
    name: str
    description: str
    parameters: Dict[str, Any]
    invoke_func: callable

    async def ainvoke(self, input_dict: Dict[str, Any]) -> Any:
        return await self.invoke_func(input_dict.get("query", ""))

# Custom HTTP client for JSON-RPC
async def send_jsonrpc_request(url: str, method: str, params: Dict = None, request_id: int = 1) -> Dict:
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    payload = {
        "jsonrpc": "2.0",
        "method": method,
        "id": request_id
    }
    if params:
        payload["params"] = params

    async with aiohttp.ClientSession() as session:
        for attempt in range(3):
            try:
                async with session.post(url, json=payload, headers=headers, timeout=10) as response:
                    if response.status == 200:
                        data = await response.json()
                        if "error" in data:
                            raise Exception(f"JSON-RPC error: {data['error']}")
                        return data.get("result")
                    else:
                        print(f"Attempt {attempt + 1} failed: HTTP {response.status}")
                        if attempt < 2:
                            await asyncio.sleep(3)
            except Exception as e:
                print(f"Attempt {attempt + 1} failed: {e}")
                if attempt < 2:
                    await asyncio.sleep(3)
        raise Exception(f"Failed to send JSON-RPC request after 3 attempts")

# Load and test MCP tools
async def load_mcp_tools():
    try:
        schema = await send_jsonrpc_request(f"{public_url}", "get_tool_schema")
        print(f"Loaded tool schema: {schema}")
        tool = SimpleTool(
            name=schema["name"],
            description=schema["description"],
            parameters=schema["parameters"],
            invoke_func=lambda query: send_jsonrpc_request(f"{public_url}", "google_serper_search", {"query": query})
        )
        return [tool]
    except Exception as e:
        print(f"Failed to load MCP tools: {e}")
        return []

# tools = asyncio.run(load_mcp_tools()) # Commented out asyncio.run
# print(f"Loaded tools: {[tool.name for tool in tools]}")

async def runGSC():
    global tools
    tools = await load_mcp_tools()
    print(f"Loaded tools: {[tool.name for tool in tools]}")
toals=None
await runGSC()

Loaded tool schema: {'name': 'google_serper_search', 'description': 'Search the web using Google Serper API and return relevant URLs.', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}}
Loaded tools: ['google_serper_search']


In [13]:
# Base URLs for filtering
base_urls = [
    "aalas.org", "jstor.org", "avmajournals.avma.org", "animalmicrobiome.biomedcentral.com",
    "irishvetjournal.biomedcentral.com", "bmcvetres.biomedcentral.com", "animaldiseases.biomedcentral.com",
    "actavetscand.biomedcentral.com", "sciendo.com", "canadianveterinarians.net", "parasite-journal.org",
    "sciencedirect.com", "microbiologyresearch.org", "jvas.in", "vaajournal.org", "frontiersin.org",
    "lp.thieme.de", "mdpi.com", "vetsci.org", "academic.oup.com", "journals.sagepub.com", "link.springer.com",
    "tandfonline.com", "onlinelibrary.wiley.com", "efsa.onlinelibrary.wiley.com",
    "resjournals.onlinelibrary.wiley.com", "beva.onlinelibrary.wiley.com", "bvajournals.onlinelibrary.wiley.com"
]

In [24]:
# Search for relevant URLs using MCP tool
question = "How to prevent Rabies?"
combined_queries = [f"{url}: {question}" for url in base_urls]
combined_queries

['aalas.org: How to prevent Rabies?',
 'jstor.org: How to prevent Rabies?',
 'avmajournals.avma.org: How to prevent Rabies?',
 'animalmicrobiome.biomedcentral.com: How to prevent Rabies?',
 'irishvetjournal.biomedcentral.com: How to prevent Rabies?',
 'bmcvetres.biomedcentral.com: How to prevent Rabies?',
 'animaldiseases.biomedcentral.com: How to prevent Rabies?',
 'actavetscand.biomedcentral.com: How to prevent Rabies?',
 'sciendo.com: How to prevent Rabies?',
 'canadianveterinarians.net: How to prevent Rabies?',
 'parasite-journal.org: How to prevent Rabies?',
 'sciencedirect.com: How to prevent Rabies?',
 'microbiologyresearch.org: How to prevent Rabies?',
 'jvas.in: How to prevent Rabies?',
 'vaajournal.org: How to prevent Rabies?',
 'frontiersin.org: How to prevent Rabies?',
 'lp.thieme.de: How to prevent Rabies?',
 'mdpi.com: How to prevent Rabies?',
 'vetsci.org: How to prevent Rabies?',
 'academic.oup.com: How to prevent Rabies?',
 'journals.sagepub.com: How to prevent Rabies?

In [32]:
article_urls = set()
results = None

for query in tqdm(combined_queries, desc="Searching URLs"):

    if tools:
        async def querySrc():
            global results
            results = await tools[0].ainvoke({"query": query})  # Use await directly
        await querySrc()
    else:
        async def querySrc():
            global results
            results = await google_serper_search(query)  # Use await directly
        await querySrc()

    for item in results:
        if "error" not in item and any(base in item["link"] for base in base_urls):
            article_urls.add(item["link"])

urls = list(article_urls)[:20]
print(f"Total unique article URLs fetched: {len(urls)}")

Searching URLs: 100%|██████████| 28/28 [00:27<00:00,  1.03it/s]

Total unique article URLs fetched: 20


In [34]:
# Load documents
loader = UnstructuredURLLoader(urls=urls)
documents = loader.load()

ERROR:langchain_community.document_loaders.url:Error fetching or processing https://www.jvas.in/article/influence-of-maternal-antibodies-on-first-vaccination-of-pups-against-rabies/pdf, exception: partition_pdf() is not available because one or more dependencies are not installed. Use: pip install "unstructured[pdf]" (including quotes) to install the required dependencies


In [35]:
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")

Total chunks: 212


In [36]:
# Create vector store
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

/tmp/ipython-input-792479809.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [37]:
# Create retriever
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"lambda_mult": 0.3})

In [38]:
# Define the RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    """Use the following context and search results to answer the question:

Context: {context}

Search Results: {search_results}

Question: {question}

Answer concisely and accurately."""
)

In [45]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Use MCP tool or fallback
async def search_tool_func(query):
    if tools:
        return await google_serper_search({"query": query})#tools[0].ainvoke
    return await google_serper_search(query)

search_tool = lambda x: asyncio.run(search_tool_func(x))

In [46]:
from langchain_core.runnables import RunnableLambda

# Define the RAG chain
rag_chain = (
    {
        "context": retriever | (lambda docs: "\n".join([doc.page_content for doc in docs])),
        "search_results": RunnableLambda(search_tool) | (lambda results: "\n".join([f"{r['title']}: {r['link']}" for r in results if "error" not in r])),
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [47]:
# Test the RAG chain
async def test_rag():
    result = await rag_chain.ainvoke(question)
    print("Answer:\n", result)

# Run the test
async def main_test_rag():
    try:
        await test_rag()
    finally:
        # Assuming public_url is defined and accessible
        ngrok.disconnect(public_url)

await main_test_rag()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:
 To prevent rabies, the following measures should be taken:

1. **Vaccination**: Ensure that pets, especially dogs, are vaccinated against rabies. Community vaccination programs can help increase coverage.

2. **Post-Exposure Prophylaxis (PEP)**: Access to PEP should be prompt and widely available for individuals who have been bitten or scratched by potentially rabid animals.

3. **Public Awareness**: Educate communities about the risks of rabies, how it is transmitted, and the importance of seeking medical attention after animal bites.

4. **Animal Control**: Implement measures to control stray animal populations and reduce the risk of rabies transmission.

5. **Monitoring and Surveillance**: Conduct regular monitoring of wildlife and domestic animals for rabies to identify and respond to outbreaks quickly.

By combining these strategies, rabies can be effectively prevented and controlled.
